In [1]:
import numpy as np 
import pandas as pd 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from gtda.time_series import SlidingWindow
from gtda.diagrams import PersistenceEntropy, Scaler
from gtda.homology import VietorisRipsPersistence
from gtda.metaestimators import CollectionTransformer
from gtda.pipeline import Pipeline
from gtda.time_series import SlidingWindow
from gtda.time_series import TakensEmbedding, SingleTakensEmbedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from ripser import Rips
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import FunctionTransformer
from ripser import Rips
from kmapper import KeplerMapper, Cover
from sklearn.cluster import KMeans
import kmapper as km
from scipy.signal import find_peaks
from gtda.plotting import plot_diagram


In [2]:
def print_scores(fitted_model):
    res = {
        "Accuracy on train:": accuracy_score(fitted_model.predict(X_train), y_train),
        "ROC AUC on train:": roc_auc_score(
            y_train, fitted_model.predict_proba(X_train)[:, 1]
        ),
        "Accuracy on valid:": accuracy_score(fitted_model.predict(X_valid), y_valid),
        "ROC AUC on valid:": roc_auc_score(
            y_valid, fitted_model.predict_proba(X_valid)[:, 1]
        ),
        "Precision 1 on train:": precision_score(fitted_model.predict(X_train), y_train, pos_label=1),
        "Precision 0 on train:": precision_score(fitted_model.predict(X_train), y_train, pos_label=0),
        "Precision 1 on valid:": precision_score(fitted_model.predict(X_valid), y_valid, pos_label=1),
        "Precision 0 on valid:": precision_score(fitted_model.predict(X_valid), y_valid, pos_label=0),
    }

    for k, v in res.items():
        print(k, round(v, 3))


## DataFrame processing

In [3]:
df = pd.read_csv('datos_agricolas_tda.csv')

C:\Users\areba\AppData\Local\Temp\ipykernel_76924\3585308298.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datos_agricolas_tda.csv')


In [4]:
df.head()

,fecha,precio_frambuesa,volumen_frambuesa,calidad_frambuesa,precio_aguacate,volumen_aguacate,calidad_aguacate,precio_chile,volumen_chile,calidad_chile,temperatura,humedad,estrategia_broker,fase_mercado,precio_fresa,serie_id,has_market_change
0,2023-01-01,70.559162,215.035239,0.731640,1.528536e+07,2610.302997,0.974015,1.657639e+10,3192.726363,0.6,22.400551,78.258721,neutral,neutro,72.624065,con_cambio_1,1
1,2023-01-02,70.580761,215.244061,0.804255,1.521303e+06,2156.506668,0.926300,7.465603e+09,3589.750248,0.6,24.774556,76.956060,neutral,neutro,72.646297,con_cambio_1,1
2,2023-01-03,70.831979,208.430465,0.512311,3.315110e+09,3383.748924,0.930883,1.190071e+10,3138.145739,0.6,25.135692,84.285750,neutral,neutro,72.904866,con_cambio_1,1
3,2023-01-04,71.125082,218.953358,0.500000,4.375625e+08,2725.801187,0.980887,3.394436e+09,2922.758055,0.6,21.448750,86.859720,neutral,neutro,73.206547,con_cambio_1,1
4,2023-01-05,70.522273,216.709261,0.500000,1.729999e+07,3081.282039,0.910412,1.921586e+09,4544.378775,0.6,24.625831,86.384635,neutral,neutro,72.586097,con_cambio_1,1


In [5]:
# Inicializamos listas
X = []
y = []

# Agrupamos por serie_id
for serie_id, group in df.groupby('serie_id'):
    # Ordenamos por dia por si acaso
    group = group.sort_values('fecha')
    
    # Extraemos la serie temporal (precio_frambuesa)
    serie_precio = group['precio_frambuesa'].values
    
    # Guardamos en X
    X.append(serie_precio)
    
    # La etiqueta es has_market_change (es constante dentro de la serie)
    etiqueta = group['has_market_change'].iloc[0]
    y.append(etiqueta)


## Takens Embedding Modelling

In [6]:
embedding_dimension = 5
embedding_time_delay = 5
stride = 2

embedder = TakensEmbedding(time_delay=embedding_time_delay,
                           dimension=embedding_dimension,
                           stride=stride)

batch_pca = CollectionTransformer(PCA(n_components=3), n_jobs=-1)

persistence = VietorisRipsPersistence(homology_dimensions=[0, 1, 2], n_jobs=-1)

scaling = Scaler()

entropy = PersistenceEntropy(normalize=True, nan_fill_value=-10)


steps_te = [("embedder", embedder),
         ("pca", batch_pca),
         ("persistence", persistence),
         ("scaling", scaling),
         ("entropy", entropy)]
topological_transformer_te = Pipeline(steps_te)

In [7]:
all_series_te = topological_transformer_te.fit_transform(X)

In [8]:
X_takens = all_series_te

In [9]:
df_processed = pd.DataFrame({
    'y': [],                     # Columna para etiquetas (vacía inicialmente)
    'takens_1': [],              # Primera componente de takens
    'takens_2': [],              # Segunda componente de takens
    'takens_3': [],              # Tercera componente de takens
    'sw_1': [],                  # Primera componente de sw
    'sw_2': [],                  # Segunda componente de sw
    'sw_3': []                   # Tercera componente de sw
})

# Asignar los valores de X_te a las columnas takens_1, takens_2, takens_3
df_processed['takens_1'] = X_takens[:, 0]  # Todas las filas, primera columna de X_te
df_processed['takens_2'] = X_takens[:, 1]  # Todas las filas, segunda columna de X_te
df_processed['takens_3'] = X_takens[:, 2]  # Todas las filas, tercera columna de X_te


In [10]:
df_processed['y'] = y 

In [11]:
df_processed.head()

,y,takens_1,takens_2,takens_3,sw_1,sw_2,sw_3
0,1,1.149604,1.710683,18.956653,NaN,NaN,NaN
1,1,1.152241,1.726879,-3.795363,NaN,NaN,NaN
2,1,1.147106,1.689278,-8.641132,NaN,NaN,NaN
3,1,1.158065,1.737775,-2.996322,NaN,NaN,NaN
4,1,1.147902,1.632653,-19.782716,NaN,NaN,NaN


In [12]:

X_train, X_valid, y_train, y_valid = train_test_split(X_takens, y, test_size=0.2, random_state=42, stratify=y)


model_takens = LogisticRegression()
model_takens.fit(X_train, y_train)
print_scores(model_takens)

Accuracy on train: 1.0
ROC AUC on train: 1.0
Accuracy on valid: 1.0
ROC AUC on valid: 1.0
Precision 1 on train: 1.0
Precision 0 on train: 1.0
Precision 1 on valid: 1.0
Precision 0 on valid: 1.0


## SlidingWindows Modelling

In [7]:
# Parámetros
window_size = 30
stride = 10

# Pasos del pipeline
steps_sw = [
    ("window", CollectionTransformer(SlidingWindow(size=window_size, stride=stride))),
    ("pca", CollectionTransformer(PCA(n_components=3), n_jobs=-1)),
    ("persistence", VietorisRipsPersistence(homology_dimensions=[0, 1, 2], n_jobs=-1)),
    ("scaling", Scaler()),
    ("entropy", PersistenceEntropy(normalize=True, nan_fill_value=-10))
]

topological_transformer_sw = Pipeline(steps_sw)


In [15]:
all_series_sw = topological_transformer_sw.fit_transform(X)

In [16]:
X_sw = all_series_sw

In [23]:
X_sw.shape

(100, 3)

In [17]:
df_processed['sw_1'] = X_sw[:, 0]
df_processed['sw_2'] = X_sw[:, 1]
df_processed['sw_3'] = X_sw[:, 2]


In [18]:
df_processed.head()

,y,takens_1,takens_2,takens_3,sw_1,sw_2,sw_3
0,1,1.149604,1.710683,18.956653,1.133404,11.210788,-10.0
1,1,1.152241,1.726879,-3.795363,1.135983,3.859877,-10.0
2,1,1.147106,1.689278,-8.641132,1.128771,4.251019,-10.0
3,1,1.158065,1.737775,-2.996322,1.151244,11.342427,-10.0
4,1,1.147902,1.632653,-19.782716,1.121861,3.111712,-0.0


In [22]:
df_processed.to_csv('datos_procesados_tda.csv')

In [20]:

X_train, X_valid, y_train, y_valid = train_test_split(X_sw, y, test_size=0.2, random_state=42)


model_sw = LogisticRegression()
model_sw.fit(X_train, y_train)
print_scores(model_sw)

Accuracy on train: 0.912
ROC AUC on train: 0.949
Accuracy on valid: 0.8
ROC AUC on valid: 0.781
Precision 1 on train: 0.947
Precision 0 on train: 0.881
Precision 1 on valid: 0.833
Precision 0 on valid: 0.75


## Mapper

In [ ]:
X_mapper = df[['precio_frambuesa', 'volumen_frambuesa']].values


In [ ]:
# Crear objeto Mapper
mapper = KeplerMapper(verbose=1)

# Parámetros
n_cubes = 4
perc_overlap = 0.43
n_clusters = 3

# Aplicar Mapper
graph = mapper.map(X_mapper, 
                   cover=Cover(n_cubes=n_cubes, perc_overlap=perc_overlap),
                   clusterer=KMeans(n_clusters=n_clusters))

# Visualizar
mapper.visualize(graph, 
                 path_html="mercado_agricola_mapper.html",
                 title="Análisis Topológico de Mercado Agrícola")


KeplerMapper(verbose=1)
Mapping on data shaped (100000, 2) using lens shaped (100000, 2)

Creating 16 hypercubes.


c:\Users\areba\anaconda3\envs\tda-env\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\areba\anaconda3\envs\tda-env\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\areba\anaconda3\envs\tda-env\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\areba\anaconda3\envs\tda-env\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` wi


Created 151 edges and 48 nodes in 0:00:02.434769.
Wrote visualization to: mercado_agricola_mapper.html


'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Análisis Topológico de Mercado Agrícola | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  font

## Takens Homología

In [8]:
X[0].shape

(1000,)

In [ ]:
embedding_dimension = 5
embedding_time_delay = 5
stride = 2
x = X[0]


# Instanciar embedding
takens = SingleTakensEmbedding(time_delay=embedding_time_delay,
                               dimension=embedding_dimension,
                               stride=stride)

# PASAR x directamente (NO reshape)
X_embedded = takens.fit_transform(x)

# Vietoris-Rips
vr = VietorisRipsPersistence(homology_dimensions=[0, 1, 2])
# Ahora sí reshape a (1, n_points, d) para VietorisRipsPersistence
X_embedded = X_embedded[None, :, :]

# Calcular persistencia
diagrams = vr.fit_transform(X_embedded)

# Plot
plot_diagram(diagrams[0])



In [22]:
def plot_persistent_homology(
    x, 
    embedding_dimension=5, 
    embedding_time_delay=5, 
    stride=2, 
    homology_dimensions=[0, 1, 2]
):
    """
    Calcula y plotea el diagrama de persistencia (H0, H1, H2) de una serie de tiempo.

    Parámetros:
    - x: np.array, serie de tiempo unidimensional.
    - embedding_dimension: dimensión del embedding de Takens.
    - embedding_time_delay: delay entre componentes del embedding.
    - stride: stride del embedding.
    - homology_dimensions: lista de dimensiones homológicas a calcular.

    """
    # Takens embedding
    takens = SingleTakensEmbedding(time_delay=embedding_time_delay,
                                   dimension=embedding_dimension,
                                   stride=stride)
    X_embedded = takens.fit_transform(x)

    # Vietoris-Rips
    vr = VietorisRipsPersistence(homology_dimensions=homology_dimensions)
    X_embedded_batch = X_embedded[None, :, :]  # reshape para batch

    diagrams = vr.fit_transform(X_embedded_batch)

    # Plot
    fig = plot_diagram(diagrams[0])
    fig.show()


In [23]:
plot_persistent_homology(X[50])